In [11]:
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    spectrograms_dir = "/content/drive/MyDrive/audio_representations/RAVDESS/spectrograms"
else :
    spectrograms_dir = "audio_representations/spectrograms"

Mounted at /content/drive


In [3]:
images = []
labels = []

for file_name in os.listdir(spectrograms_dir):
    if file_name.endswith(".png"):
        file_path = os.path.join(spectrograms_dir, file_name)

        # Convert to numpy
        img = Image.open(file_path).convert("RGB")
        img = img.resize((256, 256))
        img_array = np.array(img)

        images.append(img_array)

        label = int(file_name.split("-")[2]) - 1
        labels.append(label)

images = np.array(images)
labels = np.array(labels)

# Normalize
images = images / 255.0

# One hot
num_classes = len(np.unique(labels))
labels_one_hot = to_categorical(labels, num_classes=num_classes)

X_train, X_temp, y_train, y_temp = train_test_split(images, labels_one_hot, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Ensemble d'entraînement : {X_train.shape}, {y_train.shape}")
print(f"Ensemble de validation : {X_val.shape}, {y_val.shape}")
print(f"Ensemble de test : {X_test.shape}, {y_test.shape}")

Ensemble d'entraînement : (1008, 256, 256, 3), (1008, 8)
Ensemble de validation : (216, 256, 256, 3), (216, 8)
Ensemble de test : (216, 256, 256, 3), (216, 8)


In [4]:
# Transform 2D images into 2D sequences for LSTM
# X_train.shape[1] = 256 (height), X_train.shape[2] = 256 (width), 3 (color channels)
X_train_lstm = X_train.reshape(X_train.shape[0], X_train.shape[1], -1)  # 256, 256*3
X_val_lstm = X_val.reshape(X_val.shape[0], X_val.shape[1], -1)  # 256, 256*3
X_test_lstm = X_test.reshape(X_test.shape[0], X_test.shape[1], -1)  # 256, 256*3

In [5]:
def create_lstm_model(input_shape, num_classes):
    model = Sequential()

    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

input_shape = (256, 256 * 3)  # 256 timesteps, 256*3 caractéristiques (RGB)
model = create_lstm_model(input_shape, num_classes)

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 256, 128)            │         459,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │             520 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 513,352 (1.96 MB)

 Trainable params: 513,352 (1.96 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(
    X_train_lstm, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_lstm, y_val)
)

test_loss, test_accuracy = model.evaluate(X_test_lstm, y_test)
print(f"Loss on the test set: {test_loss}")
print(f"Accuracy on the test set: {test_accuracy}")

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.1609 - loss: 2.0654 - val_accuracy: 0.1852 - val_loss: 2.0662
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 35s 840ms/step - accuracy: 0.2002 - loss: 2.0090 - val_accuracy: 0.1667 - val_loss: 2.0085
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 42s 870ms/step - accuracy: 0.2537 - loss: 1.9504 - val_accuracy: 0.2176 - val_loss: 1.9804
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 886ms/step - accuracy: 0.2413 - loss: 1.9288 - val_accuracy: 0.2315 - val_loss: 1.9423
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 41s 876ms/step - accuracy: 0.2984 - loss: 1.8695 - val_accuracy: 0.2407 - val_loss: 1.9472
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 880ms/step - accuracy: 0.2825 - loss: 1.8605 - val_accuracy: 0.2593 - val_loss: 1.9216
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 39s 811ms/step - accuracy: 0.2874 - loss: 1.8031 - val_accuracy: 0.2454 - val_loss: 1.9184
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 861ms/step - accuracy: 0.3430 - loss: 1.7449 - val_accurac

In [7]:
save_path = '/content/drive/MyDrive/models/lstm_ravdess_spectrograms'

model.save(save_path + "/weights.h5")

In [8]:
plt.figure(figsize=(6, 5))

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Loss Evolution")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig(os.path.join(save_path, "loss_curve.png"))
plt.close()

plt.figure(figsize=(6, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Accuracy Evolution")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig(os.path.join(save_path, "accuracy_curve.png"))
plt.close()


In [12]:
emotion_labels = [
    "Neutral", "Calm", "Happy", "Sad", "Angry",
    "Fearful", "Disgust", "Surprised"
]

X_test_lstm = X_test.reshape(X_test.shape[0], X_test.shape[1], -1)

y_pred = model.predict(X_test_lstm)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

disp = ConfusionMatrixDisplay(
    confusion_matrix=conf_matrix,
    display_labels=emotion_labels
)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title("Confusion Matrix LSTM on Spectrograms")
conf_matrix_path = os.path.join(save_path, "confusion_matrix.png")
plt.savefig(conf_matrix_path)
plt.close()


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 249ms/step
